In [7]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import pandas as pd
import random
import sys
import io
import glob
import os

In [18]:
path = r'../crawler/1980s'                     # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

In [48]:
text = concatenated_df['lyric'].to_string(index=False)

In [49]:
len(text) #character length

662999

In [50]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 832


In [51]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 220987


In [52]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [56]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [58]:

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=5,
          callbacks=[print_callback])

Epoch 1/5
220987/220987 [==============================] - 270s 1ms/step - loss: 1.8976

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "그...
희미한 가로등 불빛 아래서 쓸쓸히 그대를 그리며 서있네 지나간 "
그...
희미한 가로등 불빛 아래서 쓸쓸히 그대를 그리며 서있네 지나간 어둠이 서...
그대 나를 두고 떠나버린 마음이 되어 그냥 나를 보아파 그대 내곁에 다가와 ...
그대의 눈빛은 떠나갔네 어디로 가는 나는 나의 모습  우리의 사랑이 아니다 ...
그대 나를 두고 떠나요 어느날 그대는 내 곁에 있어  그대는 사랑한다 말을 못하네    사랑한다는 ...
그대 나를 두고 떠나요 어오는                                   NaN
                                              NaN
그대 나를 두고 떠나보던 그이 아름다운 사랑이 서어 그대의 모습 사랑하고 싶어 그대 ...
그대의 눈빛은 그대 눈길에 말을  나는 사랑을 알고 있나요 아직도 그대 떠나가는 나의 마음  그렇게 바라보는 ...
                   
----- diversity: 0.5
----- Generating with seed: "그...
희미한 가로등 불빛 아래서 쓸쓸히 그대를 그리며 서있네 지나간 "
그...
희미한 가로등 불빛 아래서 쓸쓸히 그대를 그리며 서있네 지나간 아직 나의 모습을  우리...
그대는 나는 너의 마음 속에 못한 ...
                                          NaN
그대의 모습  거리  내 곁에 있어  그런의 모습을 사랑한다는 사랑한다는  헤어지면 어 그대 떠...
사랑했던 사람들 사랑한다 말 못하고 사랑하다고 걸어보린 눈물이 고 너의 그 사랑 야...
                

In [59]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [60]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

NameError: name 'model_from_json' is not defined